In [1]:
# read list of nuclear charges, give energy predictions up to nth order

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed',)).History will not be written to the database.


In [14]:
import numpy as np
import pandas as pd
import qml
import scipy.spatial as scs

In [3]:
def get_predictions(comb = [6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 5, 7]):
    rho, dsingle, dneigh1, dneigh2, dneigh3, dneigh4, dneigh5 = read_densities()
    grid_points, grid_weights = read_grid()
    
    E = 0. # base energy
    deltaZ = np.array(comb) - 6
    
    # 0-th order, no rotation necessary
    for idx, Z in enumerate(comb):
        if deltaZ[idx] == 0:
            continue
        ds = np.linalg.norm(grid_points - c.coordinates[idx], axis=1)
        E += np.sum(deltaZ[idx] * rho * grid_weights / ds)
    
    # 1st order
    for idx, Z in enumerate(comb):
        if deltaZ[idx] == 0:
            continue
        mapping = build_reindexing_1(idx, 0)
        for j in range(20): 
            if deltaZ[mapping[j]] == 0:
                continue
            ds = np.linalg.norm(grid_points - c.coordinates[mapping[j]], axis=1)
            E += np.sum(deltaZ[mapping[j]] * dsingle * grid_weights / ds)
    
    # 2nd order
    classify distance, choose density accordigly, same logic as before
    # 0-1 neigh1
    # 0-2 neigh2
    # 0-8 neigh3
    # 0-10 neigh4
    # 0-16 neigh5

[6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6]

In [6]:
c = qml.Compound('../../test/c20.xyz')

In [10]:
np.linalg.norm(c.coordinates[0] - c.coordinates[1]), np.linalg.norm(c.coordinates[0] - c.coordinates[2]), np.linalg.norm(c.coordinates[0] - c.coordinates[8]), np.linalg.norm(c.coordinates[0] - c.coordinates[10]), np.linalg.norm(c.coordinates[0] - c.coordinates[16])

(1.4941261703959974,
 2.4175469174481705,
 3.418926705229528,
 3.9116718460879882,
 4.187313133010966)

In [89]:
def _get_nn(refsite):
    distances = np.linalg.norm(c.coordinates - c.coordinates[refsite], axis=1)
    return np.argsort(distances)[1:4]
def build_reindexing_1(refsite, ontosite):
    cog = np.mean(c.coordinates, axis=0)
    valid = False
    for Ann in _get_nn(refsite):
        for Bnn in _get_nn(ontosite):
            A = c.coordinates[[refsite, Ann]]
            B = c.coordinates[[ontosite, Bnn]]
            rot = scs.transform.Rotation.match_vectors(A, B)[0]
            transformed = rot.apply(c.coordinates)
            found = []
            for site in range(len(c.coordinates)):
                ds = np.linalg.norm(transformed - c.coordinates[site], axis=1)
                if min(ds) < 1e-5:
                    found.append(np.argmin(ds))
            if set(found) == set([_ for _ in range(len(c.coordinates))]) and found[refsite] == ontosite:
                valid = True
                break
        if valid:
            break
    if not valid:
        raise ValueError('no solution')
    return found

build_reindexing_1(1, 3)

[2, 3, 4, 0, 1, 9, 10, 11, 12, 13, 14, 5, 6, 7, 8, 17, 18, 19, 15, 16]

In [91]:
def build_reindexing_2(refsite1, refsite2, ontosite1, ontosite2):
    cog = np.mean(c.coordinates, axis=0)
    valid = False
    A = c.coordinates[[refsite1, refsite2]]
    B = c.coordinates[[ontosite1, ontosite2]]
    rot = scs.transform.Rotation.match_vectors(A, B)[0]
    transformed = rot.apply(c.coordinates)
    found = []
    for site in range(len(c.coordinates)):
        ds = np.linalg.norm(transformed - c.coordinates[site], axis=1)
        if min(ds) < 1e-5:
            found.append(np.argmin(ds))
    if set(found) == set([_ for _ in range(len(c.coordinates))]) and found[refsite1] == ontosite1 and found[refsite2] == ontosite2:
        pass
    else:
        raise ValueError('no solution')
    return found
build_reindexing_2(0, 5, 7, 1)

[7, 6, 19, 15, 8, 1, 0, 5, 14, 18, 17, 16, 10, 9, 2, 13, 12, 11, 3, 4]